In [ ]:
from multiprocessing import Pool

import os
import inspect

import numpy as np
import time

import pandas as pd

In [ ]:
def parallel_process(functions, args, libraries=[], out_name='_TMP_parallel_process_'):
    path = './resources/script/'
    if not os.path.exists(path):
        os.makedirs(path)
    
    # Export libraries & functions which needs to be run parallelly to .py script
    out_path = f'{path}{out_name}.py'
    with open(out_path, 'w') as file:
        # Libraries
        for index, lib in enumerate(libraries):
            file.write(f'{lib}\n')
            if index == len(libraries) -1:
                file.write('\n\n\n')
        
        # Functions
        for index, fn in enumerate(functions):
            if index == len(functions) -1:
                file.write(inspect.getsource(fn).replace(fn.__name__, 'parallel_fn'))
            else:
                file.write(inspect.getsource(fn))
                file.write('\n\n\n')
    
    # Import exported functions to run parallelly, or else jupyter notebook will not be able to run it
    from importlib import import_module
    module = import_module(f'resources.script.{out_name}')
    
    p = Pool()
    result = p.starmap(module.parallel_fn, args)
    p.close()
    p.join()
    
    return result

In [ ]:
def fn(x, tmp_dict):
    time.sleep(.1) # simulate heavy processing
    
    try:
        return tmp_dict[x] * x
    except:
        return x * x

In [ ]:
ndarrays = np.arange(60).reshape(3, 4, 5)
ndarrays

In [ ]:
tmp_dict = dict()
tmp_dict[10] = 100
tmp_dict[20] = 200
tmp_dict[30] = 300
tmp_dict[40] = 400
tmp_dict[50] = 500

tmp_dict

In [ ]:
# Without Parallel
EXEC_START = time.time()

for ndarray in ndarrays:
    for array in ndarray:
        result = []
        for x in array:
            result.append(fn(x, tmp_dict))
        print(result)
    print()
    
EXEC_END = time.time()
print(f'{EXEC_END - EXEC_START} sec.')

In [ ]:
# Parallel (1 Loop)
EXEC_START = time.time()

for ndarray in ndarrays:
    for array in ndarray:
        result = parallel_process(functions=[fn],
                                  args=[(x, tmp_dict) for x in array],
                                  libraries=['import time'])
        print(result)
    print()
    
EXEC_END = time.time()
print(f'{EXEC_END - EXEC_START} sec.')

In [ ]:
# Parallel (All Loops)
EXEC_START = time.time()

result = parallel_process(functions=[fn],
                          args=[(x, tmp_dict) for x in ndarrays.reshape(-1,)],
                          libraries=['import time'])
# print(result)
print(np.array(result).reshape(-1, 4, 5))

EXEC_END = time.time()
print(f'\n{EXEC_END - EXEC_START} sec.')